#### Importing Modules

In [ ]:
!pip install --upgrade tensorflow-federated

In [ ]:
import os

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from collections import OrderedDict

import tensorflow as tf
import tensorflow_federated as tff

In [ ]:
folder_path = os.path.join('..', 'dataset', 'processed')
flight_icao_num = 'a007c6'

#### Reading the Dataset

In [ ]:
def read_data():
    dfs = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)

            dfs.append(df)

    combined_df = pd.concat(dfs, ignore_index=True)

    columns_to_drop = ['signature_label', 'icao24', 'Unnamed: 0']

    y = combined_df['signature_label']
    X = combined_df.drop(columns_to_drop, axis=1)

    return X, y

In [ ]:
def get_train_test_split(X, y, trip_numbers: list):
    X_train = pd.DataFrame()
    X_test = pd.DataFrame()

    all_files = os.listdir(folder_path)

    files_not_in_trip_numbers = [file for file in all_files if not any((flight_icao_num + "_" + str(trip) + ".csv") in file for trip in trip_numbers)]
    files_in_trip_numbers = [file for file in all_files if any((flight_icao_num + "_" + str(trip) + ".csv") in file for trip in trip_numbers)]

    print(files_in_trip_numbers)

    for train_data_file in files_in_trip_numbers:
        file_path = os.path.join(folder_path, train_data_file)
        
        print("TRAIN", file_path)

        df = pd.read_csv(file_path)
        X_train = pd.concat([X_train, df], ignore_index=True)

    for test_data_file in files_not_in_trip_numbers:
        file_path = os.path.join(folder_path, test_data_file)
        
        print("TEST", file_path)

        df = pd.read_csv(file_path)
        X_test = pd.concat([X_test, df], ignore_index=True)

    drop_columns = ['Unnamed: 0', 'icao24', 'trip_number']
    X_train = X_train.drop(drop_columns, axis=1)
    X_test = X_test.drop(drop_columns, axis=1)

    y_train = X_train['signature_label']
    X_train = X_train.drop('signature_label', axis=1)

    y_test = X_test['signature_label']
    X_test = X_test.drop('signature_label', axis=1)

    return X_train, X_test, y_train, y_test

#### Standard Scalar

In [ ]:
def perform_scaling(input_X):
    scaler = StandardScaler()

    scaled_value = scaler.fit_transform(input_X)
    return scaled_value

#### Converting to Tensors Format

In [ ]:
def convert_to_tensors(input_features, labels):
    X_inputs = tf.convert_to_tensor(input_features, name="flights-inputs")
    y_labels = tf.convert_to_tensor(labels, name="flights-labels")

    return X_inputs, y_labels

#### Making Fed Learning Scenario

In [ ]:
def create_fed_dataset(train, test):
    client_data = {}

    client_data[f"f_1"] = train
    client_data[f"f_2"] = test

    # client_ids = list(client_data.keys())

    client_data_values = []

    for key, value in client_data.items():
        client_data_values.append(value)

    return client_data_values

In [ ]:
def model_fn():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(4, )),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    return tff.learning.from_keras_model(
      model,
      input_spec=input_spec_value,
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
      metrics=[tf.keras.metrics.Accuracy()])

In [ ]:
X, y = read_data()

X_train, X_test, y_train, y_test = get_train_test_split(X, y, trip_numbers=[0,1])

X_train_scaled = perform_scaling(X_train)
X_test_scaled = perform_scaling(X_test)

X_train_raw = np.asarray(X_train_scaled).astype(np.float32)[:, np.newaxis]
y_train_label = np.asarray(y_train).astype(np.int32).reshape(X_train_scaled.shape[0], 1)

X_test_raw = np.asarray(X_test_scaled).astype(np.float32)[:, np.newaxis]
y_test_label = np.asarray(y_test).astype(np.int32).reshape(X_test_scaled.shape[0], 1)

X_train_features, y_train_labels = convert_to_tensors(X_train_raw, y_train_label)
X_test_features, y_test_labels = convert_to_tensors(X_test_raw, y_test_label)

train_ds = tf.data.Dataset.from_tensor_slices((X_train_features, y_train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((X_test_features, y_test_labels))

In [ ]:
input_spec_value = train_ds.element_spec

federated_train_data = create_fed_dataset(train_ds, test_ds)

In [ ]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.1))

# trainer = tff.learning.algorithms.build_weighted_fed_avg(
#      model_fn,
#      client_optimizer_fn=lambda: tf.keras.optimizers.SGD(0.1))

In [ ]:
print(training_process.initialize.type_signature.formatted_representation())

##### Initializing

In [ ]:
train_state = training_process.initialize()

In [ ]:
train_state

In [ ]:
NUM_ROUNDS = 11

tff_train_acc = []
tff_train_loss = []
tff_val_acc = []
tff_val_loss = []

for i in range(NUM_ROUNDS):
  # Train
  result = trainer.next(train_state, federated_train_data)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  print('round {:2d}, metrics={}'.format(i, train_metrics))


  # # Validation
  # federated_metrics = tff.learning.algorithms.build_fed_eval(result.state.global_model_weights(), val_data)
  # val_metrics = federated_metrics['eval']

  # # Metrics
  # train_loss = train_metrics['loss']
  # train_acc = train_metrics['binary_crossentropy']
  # val_loss = val_metrics['loss']
  # val_acc = val_metrics['accuracy']

  # # Print
  # print('round {:2d}\ntrain_loss={l:.3f}, train_acc={ac:.3f}'.format(
  #     i+1, l=train_loss, ac=train_metrics['binary_crossentropy']))
  # print('val_loss: {:.3f} val_acc: {:.3f}'.format(
  #     val_loss, val_acc))

  # # logs
  # tff_train_acc.append(float(train_metrics['binary_crossentropy']))
  # tff_train_loss.append(float(train_metrics['loss']))
  # tff_val_acc.append(float(val_metrics['binary_crossentropy']))
  # tff_val_loss.append(float(val_metrics['loss']))
  # current_round = i